In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from scipy import stats
from statistics import mode

# Firstly, we create a dataset with 1000 rows and 3 columns (for x, y, z axes)
np.random.seed(0) # for reproducible results
data = np.random.rand(1000,3)

# We now create a dataframe from the data
columns = ['x', 'y', 'z']
df = pd.DataFrame(data, columns = columns)

# For the purpose of our example, we manually assign the labels of our dataframe
labels = ['walking']*300 + ['standing']*400 +['walking']*300

# We then add the label column to the dataframe
df['label'] = labels

# Display the dataframe
print(df)


            x         y         z    label
0    0.548814  0.715189  0.602763  walking
1    0.544883  0.423655  0.645894  walking
2    0.437587  0.891773  0.963663  walking
3    0.383442  0.791725  0.528895  walking
4    0.568045  0.925597  0.071036  walking
..        ...       ...       ...      ...
995  0.698630  0.503697  0.025738  walking
996  0.774353  0.560374  0.082494  walking
997  0.475214  0.287293  0.879682  walking
998  0.284927  0.941687  0.546133  walking
999  0.323614  0.813545  0.697400  walking

[1000 rows x 4 columns]


### Time-domain and Frequency-Domain Features

In [2]:
# Function to calculate zero crossing rate
def zero_crossing_rate_sgn(window_data):
    # Define the signum function
    sgn = np.vectorize(lambda x: -1 if x < 0 else (1 if x > 0 else 0))

    # Apply the signum function to window data
    sign_data = sgn(window_data)

    # Calculate the ZCR using the sgn function
    zcr = np.sum(np.abs(np.diff(sign_data))) / (2 * (len(window_data) - 1))
    return zcr

# Function to calculate entropy
def calculate_entropy(data):
    histogram, bin_edges = np.histogram(data, bins='auto', density=True)
    probabilities = histogram * np.diff(bin_edges)
    probabilities = probabilities[probabilities > 0]
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy



In [3]:
# Function to calculate spectral energy
def calculate_spectral_energy(fft_values):
    spectral_energy = np.sum(np.abs(fft_values)**2)
    return spectral_energy

# Function to calculate dominant frequency
def calculate_dominant_frequency(fft_vals, fft_freq):
    # Filter to keep only the positive frequencies
    positive_indices = fft_freq > 0
    positive_fft_vals = fft_vals[positive_indices]
    positive_fft_freq = fft_freq[positive_indices]

    # Calculate the Power Spectral Density (PSD) for positive frequencies
    positive_psd = np.abs(positive_fft_vals)**2

    # Identify significant frequencies
    threshold = 0.1 * np.max(positive_psd)  # Example: 10% of the max PSD value
    significant_freqs_indices = np.where(positive_psd >= threshold)[0]
    significant_freqs = positive_fft_freq[significant_freqs_indices]

    # Calculate the mean or median of these significant frequencies
    dominant_freq = np.mean(significant_freqs) if len(significant_freqs) > 0 else 0
    return dominant_freq

# Function to calculate psd
def calculate_psd(fft_vals, N):
    psd = np.abs(fft_vals)**2 / N
    return psd

# Function to calculate spectral entropy
def calculate_spectral_entropy(psd):
    normalized_psd = psd / np.sum(psd)
    spectral_entropy = -np.sum(normalized_psd * np.log2(normalized_psd))
    return spectral_entropy

# Function to calculate peak frequency
def calculate_peak_frequency(fft_vals, fft_freq):
    # Calculate the magnitude of the FFT
    magnitude = np.abs(fft_vals)

    # Find the index of the maximum magnitude
    idx_peak = np.argmax(magnitude)

    # Return the corresponding frequency
    peak_frequency = fft_freq[idx_peak]
    return peak_frequency

In [4]:
def calculate_spectral_features(fft_vals, fft_freq):
    # Compute power spectrum
    psd = np.abs(fft_vals)**2
    total_power = np.sum(psd)
    psd_normalized = psd / total_power

    # Spectral Centroid
    spectral_centroid = np.sum(fft_freq * psd_normalized)

    # Spectral Spread
    spectral_spread = np.sqrt(np.sum(((fft_freq - spectral_centroid)**2) * psd_normalized))

    # Spectral Skewness and Kurtosis
    spectral_skewness = np.sum(((fft_freq - spectral_centroid)**3) * psd_normalized) / (spectral_spread**3)
    spectral_kurtosis = np.sum(((fft_freq - spectral_centroid)**4) * psd_normalized) / (spectral_spread**4)

    # Spectral Flatness
    spectral_flatness = np.exp(np.mean(np.log(psd + 1e-10))) / np.mean(psd + 1e-10)

    return spectral_centroid, spectral_spread, spectral_skewness, spectral_kurtosis, spectral_flatness

In [5]:
# Create a function to extract time-domain features from the dataframe

def calculate_features(data, delta_t):
    # 'data' is a pandas dataframe with named columns
    # We initialize a dictionary to store the features

    features = {}
    
    # Basic statistical measures
    for col in data.columns:
        features[f'{col}_mean'] = data[col].mean()
        features[f'{col}_std_dev'] = data[col].std()
        features[f'{col}_variance'] = data[col].var()
        features[f'{col}_icv'] = features[f'{col}_mean'] / features[f'{col}_std_dev']
        features[f'{col}_median'] = data[col].median()
        features[f'{col}_minimum'] = data[col].min()
        features[f'{col}_maximum'] = data[col].max()
        features[f'{col}_skewness'] = data[col].skew()
        features[f'{col}_kurtosis'] = data[col].kurt()
        # Calculate Interquartile Range
        features[f'{col}_iqr'] = stats.iqr(data[col])
        features[f'{col}_zcr'] = zero_crossing_rate_sgn(data[col])
        features[f'{col}_peak_to_peak'] = np.max(data[col]) - np.min(data[col])
        
        # Entropy for each axis
        features[f'{col}_entropy'] = calculate_entropy(data[col])
      
    # Paiwise Correlation
    corr_matrix = data.corr().values
    for i, col1 in enumerate(data.columns):
        for j, col2 in enumerate(data.columns):
            if i<j: # to avoid duplicate pairs
                features[f'correlation_{col1}_{col2}'] = corr_matrix[i,j]
    
    # Signal Magnitude Area
    features['sma'] = data.abs().sum().sum() / len(data)
    
    # Energy calculation
    energy = np.sum(data['x']**2 + data['y']**2 + data['z']**2)
    features['energy'] = energy
    
    # Combined entropy
    combined_data = np.hstack((data['x'], data['y'], data['z']))
    features['combined_entropy'] = calculate_entropy(combined_data)  
    
    # Calculate Frequency-Domain Features
    fs = 1 / delta_t  # Frequency based on delta_t
    N = len(data)

    for col in ['x', 'y', 'z']:
        # Perform FFT
        fft_vals = np.fft.fft(data[col])
        fft_freq = np.fft.fftfreq(N, 1/fs)

        # Filter to keep only the positive frequencies
        positive_indices = fft_freq > 0
        positive_fft_vals = fft_vals[positive_indices]
        positive_fft_freq = fft_freq[positive_indices]

        # Spectral Energy
        features[f'{col}_spectral_energy'] = calculate_spectral_energy(fft_vals)
        
        # Dominant Frequency
        features[f'{col}_dominant_frequency'] = calculate_dominant_frequency(positive_fft_vals, positive_fft_freq)
        
        # Calculate PSD and Spectral Entropy
        psd = calculate_psd(fft_vals, N)
        features[f'{col}_max_psd'] = np.max(psd)  # Storing PSD values

        spectral_entropy = calculate_spectral_entropy(psd)
        features[f'{col}_spectral_entropy'] = spectral_entropy
        
        # Calculate Peak Frequency
        features[f'{col}_peak_frequency'] = calculate_peak_frequency(positive_fft_vals, positive_fft_freq)
        
        # Spectral Features Calculation
        spectral_centroid, spectral_spread, spectral_skewness, spectral_kurtosis, spectral_flatness = calculate_spectral_features(positive_fft_vals, positive_fft_freq)
        
        features[f'{col}_spectral_centroid'] = spectral_centroid
        features[f'{col}_spectral_spread'] = spectral_spread
        features[f'{col}_spectral_skewness'] = spectral_skewness
        features[f'{col}_spectral_kurtosis'] = spectral_kurtosis
        features[f'{col}_spectral_flatness'] = spectral_flatness
        
    return features
    

In [6]:
def feature_extraction_with_windows(data, window_size, step_size, calculate_features_func, delta_t):
    num_samples = len(data)
    features_list = []
    
    for start in range(0, num_samples-window_size+1, step_size):
        end = start + window_size
        window_data = data.iloc[start:end]
        window_features = calculate_features_func(window_data.drop(columns=['label']), delta_t)
        
        # Get the most frequent label in the window
        try:
            window_label = mode(window_data['label'])
        except:
            # In case there is no unique mode
            # Use the first label in case of a tie
            window_label = window_data['label'].value_counts().index[0]
            
        window_features['label'] = window_label
        features_list.append(window_features)
    
    return pd.DataFrame(features_list)

In [7]:
# Example

window_size = 64 # Number of samples in each window
step_size = 10 # Step size for the moving window
delta_t = 1/12  # Example: 12 Hz sampling rate

preprocessed_data = feature_extraction_with_windows(df,
                                                    window_size, 
                                                    step_size, 
                                                    calculate_features, delta_t)
# Check the first five rows of the data
preprocessed_data

,x_mean,x_std_dev,x_variance,x_icv,x_median,x_minimum,x_maximum,x_skewness,x_kurtosis,x_iqr,...,z_dominant_frequency,z_max_psd,z_spectral_entropy,z_peak_frequency,z_spectral_centroid,z_spectral_spread,z_spectral_skewness,z_spectral_kurtosis,z_spectral_flatness,label
0,0.485164,0.283024,0.080103,1.714214,0.557707,0.004695,0.976761,-0.117813,-1.265305,0.488665,...,2.964844,18.068964,1.896006,5.4375,3.364475,1.754826,-0.163143,1.655263,0.613534,walking
1,0.471164,0.290020,0.084111,1.624592,0.513046,0.004695,0.990339,-0.012669,-1.296235,0.498229,...,2.955882,18.049587,1.792366,5.4375,3.346931,1.897080,-0.241341,1.503240,0.444676,walking
2,0.477353,0.304979,0.093012,1.565199,0.489258,0.004695,0.990339,0.046004,-1.363893,0.529141,...,2.875000,16.818438,1.996417,4.8750,3.158512,1.821276,-0.117618,1.504796,0.618335,walking
3,0.529242,0.311065,0.096762,1.701385,0.583842,0.004695,0.990339,-0.184121,-1.336513,0.572285,...,2.894022,17.375247,2.021608,5.4375,3.148814,1.855366,-0.005481,1.529567,0.622232,walking
4,0.505857,0.322896,0.104262,1.566624,0.523476,0.011427,0.990339,-0.039249,-1.460744,0.625657,...,3.215625,16.681504,2.092557,5.4375,3.287555,1.854977,-0.112897,1.504364,0.581815,walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0.470640,0.297535,0.088527,1.581797,0.453207,0.005052,0.966221,0.100761,-1.415419,0.539741,...,2.722500,14.088993,2.196136,0.1875,2.381982,1.713564,0.325589,1.755105,0.670753,walking
90,0.460968,0.296479,0.087900,1.554810,0.380202,0.005052,0.969582,0.160188,-1.474648,0.558898,...,2.553571,13.293665,2.235285,4.8750,2.498865,1.677125,0.349108,1.816249,0.574758,walking
91,0.491957,0.313256,0.098129,1.570464,0.427072,0.005052,0.970213,0.033945,-1.509947,0.561680,...,2.769886,12.742728,2.435980,1.6875,2.506152,1.527115,0.372891,2.027139,0.585258,walking
92,0.510455,0.311997,0.097342,1.636091,0.523032,0.005052,0.989766,-0.008317,-1.491010,0.549002,...,3.044118,13.674606,2.370937,1.8750,2.649721,1.510983,0.203440,1.850801,0.449335,walking
